<a href="https://colab.research.google.com/github/e-pyrom/Build-Projects/blob/master/Build_2_Car_MSRP_Predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

# Ignore this warning: https://github.com/dmlc/xgboost/issues/4300
# xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='xgboost')

In [0]:
import pandas as pd
import numpy as np

pd.options.display.max_columns = None
cars = pd.read_csv('https://raw.githubusercontent.com/e-pyrom/Build-Projects/master/cars_v2.csv')

In [47]:
# Get Pandas Profiling Report
from pandas_profiling import ProfileReport

profile = ProfileReport(cars)
profile

/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,17
Number of observations,8253
Total Missing (%),2.0%
Total size in memory,1.1 MiB
Average record size in memory,136.0 B
Numeric,8
Categorical,8
Boolean,0
Date,0
Text (Unique),0
Rejected,1


In [48]:
cars.shape

(8253, 17)

In [49]:
# Value count for each unique value. Subset with Makes with more than 100 data points.
cars['Make'].value_counts()

Chevrolet        859
Volkswagen       754
Ford             651
Toyota           636
Nissan           469
Honda            426
Dodge            407
GMC              371
Mazda            340
Suzuki           308
Hyundai          275
Subaru           226
Infiniti         226
Kia              223
Mitsubishi       181
Volvo            170
Buick            166
Acura            165
Cadillac         161
Chrysler         159
Audi             151
Pontiac          141
BMW              141
Lincoln          119
Lexus            117
Saab              77
Mercedes-Benz     73
FIAT              62
Scion             60
Oldsmobile        57
Land Rover        56
HUMMER            17
Lotus              3
Plymouth           2
Porsche            2
Genesis            2
Name: Make, dtype: int64

In [50]:
#Dropping Makes with less than 100 data points

fewer = cars['Make'].value_counts()
cars = cars[~cars['Make'].isin(fewer[fewer < 100].index)]
cars.shape

(7842, 17)

In [51]:
cars['Make'].value_counts()

Chevrolet     859
Volkswagen    754
Ford          651
Toyota        636
Nissan        469
Honda         426
Dodge         407
GMC           371
Mazda         340
Suzuki        308
Hyundai       275
Subaru        226
Infiniti      226
Kia           223
Mitsubishi    181
Volvo         170
Buick         166
Acura         165
Cadillac      161
Chrysler      159
Audi          151
Pontiac       141
BMW           141
Lincoln       119
Lexus         117
Name: Make, dtype: int64

In [52]:
#Replaced all missing values with 'Unknown'
cars = cars.fillna('Unknown')
cars.isnull().sum()

Unnamed:_0           0
Make                 0
Model                0
Year                 0
Engine_Fuel_Type     0
Engine_HP            0
Engine_Cylinders     0
Transmission_Type    0
Driven_Wheels        0
Number_of_Doors      0
Market_Category      0
Vehicle_Size         0
Vehicle_Style        0
highway_MPG          0
city_mpg             0
Popularity           0
MSRP                 0
dtype: int64

In [53]:
cars['MSRP'].describe()

count     7842.000000
mean     30136.759628
std       8830.190968
min      10135.000000
25%      23495.000000
50%      29365.000000
75%      36400.000000
max      49995.000000
Name: MSRP, dtype: float64

In [54]:
#Define Market Categories, and fill in missing values
cars['Market_Category'].describe()

count        7842
unique         50
top       Unknown
freq         2571
Name: Market_Category, dtype: object

In [55]:
#Market Category column needs to be cleaned
cars['Market_Category'].value_counts()

Unknown                                          2571
Crossover                                        1094
Flex Fuel                                         805
Performance                                       506
Hatchback                                         458
Luxury                                            400
Luxury,Performance                                306
Crossover,Luxury                                  261
Hatchback,Performance                             227
High-Performance                                  146
Hybrid                                            117
Diesel                                             84
Hatchback,Hybrid                                   72
Crossover,Hatchback                                72
Crossover,Luxury,Performance                       66
Factory Tuner,High-Performance                     65
Luxury,High-Performance                            63
Crossover,Flex Fuel                                63
Factory Tuner,Performance   

In [56]:
#Replace high-performance with performance
cars['Market_Category'].replace({'High-Performance':'Performance'}, inplace=True)

cars['Transmission_Type'].value_counts()

#Create new columns for each unique category type
cars['Unknown'] = cars['Market_Category']== 'Unknown'
cars['Hatchback'] = cars['Market_Category']== 'Hatchback'
cars['Crossover'] = cars['Market_Category']== 'Crossover'
cars['Flex Fuel'] = cars['Market_Category']== 'Flex Fuel'
cars['Performance'] = cars['Market_Category']== 'Performance'
cars['Luxury'] = cars['Market_Category']== 'Luxury'
cars['Hybrid'] = cars['Market_Category']== 'Hybrid'
cars['Diesel'] = cars['Market_Category']== 'Diesel'
cars['Factory Tuner'] = cars['Market_Category']== 'Factory Tuner'

cars.head(5)

,Unnamed:_0,Make,Model,Year,Engine_Fuel_Type,Engine_HP,Engine_Cylinders,Transmission_Type,Driven_Wheels,Number_of_Doors,Market_Category,Vehicle_Size,Vehicle_Style,highway_MPG,city_mpg,Popularity,MSRP,Unknown,Hatchback,Crossover,Flex Fuel,Performance,Luxury,Hybrid,Diesel,Factory Tuner
0,0,BMW,1 Series M,2011,premium unleaded (required),335,6,MANUAL,rear wheel drive,2.0,"Factory Tuner,Luxury,High-Performance",Compact,Coupe,26,19,3916,46135,False,False,False,False,False,False,False,False,False
1,1,BMW,1 Series,2011,premium unleaded (required),300,6,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Convertible,28,19,3916,40650,False,False,False,False,False,False,False,False,False
2,2,BMW,1 Series,2011,premium unleaded (required),300,6,MANUAL,rear wheel drive,2.0,"Luxury,High-Performance",Compact,Coupe,28,20,3916,36350,False,False,False,False,False,False,False,False,False
3,3,BMW,1 Series,2011,premium unleaded (required),230,6,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Coupe,28,18,3916,29450,False,False,False,False,False,False,False,False,False
4,4,BMW,1 Series,2011,premium unleaded (required),230,6,MANUAL,rear wheel drive,2.0,Luxury,Compact,Convertible,28,18,3916,34500,False,False,False,False,False,True,False,False,False


In [57]:
#Drop original Market Category column
cars.drop(columns='Market_Category', inplace=True)
cars.shape

(7842, 25)

In [58]:
#Drop unneeded columns
drop_col = ['Unnamed:_0', 'Number_of_Doors', 'Popularity', 'Vehicle_Size']
cars.drop(columns=drop_col, inplace=True)
cars.shape

(7842, 21)

In [59]:
#Cleaning up Model column, high cardinality
cars['Model'].value_counts(ascending=True)

K900                       1
GS 430                     1
Accord Plug-In Hybrid      1
5 Series                   1
GX 460                     1
                        ... 
Sierra 1500               80
Beetle Convertible        89
F-150                     99
Tundra                   136
Silverado 1500           147
Name: Model, Length: 502, dtype: int64

In [60]:
cars['Model'].describe()

count               7842
unique               502
top       Silverado 1500
freq                 147
Name: Model, dtype: object

In [0]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="ticks", color_codes=True)

sns.catplot(x="Model", palette='Blues', kind="count", data=cars);
plt.show()

In [62]:
#Dropping Models with more than 50 data points, avoid overfitting on those specific models
high_card = cars['Model'].value_counts()
cars = cars[~cars['Model'].isin(high_card[high_card > 50].index)]
cars.shape

(6444, 21)

In [63]:
#Dropping Makes with less than 100 data points
cars['Model'].value_counts()

Golf GTI                  50
XC60                      50
Sierra 1500 Classic       49
Silverado 1500 Classic    49
Canyon                    48
                          ..
GX 460                     1
TTS                        1
Accord Plug-In Hybrid      1
G3                         1
K900                       1
Name: Model, Length: 484, dtype: int64

In [64]:
#Engine Fuel Type
cars['Engine_Fuel_Type'].value_counts()

regular unleaded                                4342
premium unleaded (recommended)                   923
premium unleaded (required)                      618
flex-fuel (unleaded/E85)                         448
diesel                                            57
electric                                          42
flex-fuel (unleaded/natural gas)                   6
flex-fuel (premium unleaded recommended/E85)       5
Unknown                                            3
Name: Engine_Fuel_Type, dtype: int64

In [65]:
#Replace current engine fuel types

cars['Engine_Fuel_Type'].replace({'premium unleaded (recommended)':'premium unleaded', 'premium unleaded (required)':'premium unleaded', 
                                 'flex-fuel (unleaded/E85)':'flex-fuel', 'flex-fuel (unleaded/natural gas)':'flex-fuel',
                                  'flex-fuel (premium unleaded recommended/E85)':'flex-fuel'}, inplace=True)

cars['Engine_Fuel_Type'].value_counts()

regular unleaded    4342
premium unleaded    1541
flex-fuel            459
diesel                57
electric              42
Unknown                3
Name: Engine_Fuel_Type, dtype: int64

In [66]:
#Cleaning up transmissions, combine automatic with automated_manual
cars['Transmission_Type'].value_counts()

AUTOMATIC           4918
MANUAL              1209
AUTOMATED_MANUAL     273
DIRECT_DRIVE          44
Name: Transmission_Type, dtype: int64

In [67]:
cars['Transmission_Type'].replace({'AUTOMATED_MANUAL':'AUTOMATIC'}, inplace=True)

cars['Transmission_Type'].value_counts()

AUTOMATIC       5191
MANUAL          1209
DIRECT_DRIVE      44
Name: Transmission_Type, dtype: int64

In [68]:
#Cleaning up Vehicle Style
cars['Vehicle_Style'].value_counts()

Sedan                  1817
4dr SUV                1760
4dr Hatchback           540
Wagon                   431
Coupe                   411
Passenger Minivan       279
Convertible             245
Crew Cab Pickup         217
2dr Hatchback           208
Extended Cab Pickup     185
Passenger Van           103
Regular Cab Pickup       91
Cargo Minivan            63
Cargo Van                49
2dr SUV                  30
Convertible SUV          15
Name: Vehicle_Style, dtype: int64

In [69]:
cars['Vehicle_Style'].replace({'4dr SUV':'SUV'}, inplace=True)
cars['Vehicle_Style'].replace({'4dr Hatchback':'Hatchback'}, inplace=True)
cars['Vehicle_Style'].replace({'Passenger Minivan':'Minivan'}, inplace=True)
cars['Vehicle_Style'].replace({'Crew Cab Pickup':'Pickup'}, inplace=True)
cars['Vehicle_Style'].replace({'2dr Hatchback':'Hatchback'}, inplace=True)
cars['Vehicle_Style'].replace({'Extended Cab Pickup':'Pickup'}, inplace=True)
cars['Vehicle_Style'].replace({'Passenger Van':'Minivan'}, inplace=True)
cars['Vehicle_Style'].replace({'Regular Cab Pickup':'Pickup'}, inplace=True)
cars['Vehicle_Style'].replace({'Cargo Minivan':'Cargo Van'}, inplace=True)
cars['Vehicle_Style'].replace({'2dr SUV':'SUV'}, inplace=True)
cars['Vehicle_Style'].replace({'Convertible SUV':'SUV'}, inplace=True)

cars['Vehicle_Style'].value_counts()

Sedan          1817
SUV            1805
Hatchback       748
Pickup          493
Wagon           431
Coupe           411
Minivan         382
Convertible     245
Cargo Van       112
Name: Vehicle_Style, dtype: int64

In [70]:
#Distribution of data

import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

sns.set(color_codes=True)

x = cars['MSRP']

sns.distplot(x)

In [71]:
## Random train, val, test datasets
from sklearn.model_selection import train_test_split

X, y = cars.drop(columns='MSRP'), cars['MSRP']

#train and test split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                  test_size=0.20,
                                                  random_state=42)

#train and val split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, 
                                                   test_size=0.20, random_state=42)

print('Train split:', X_train.shape, 'Val split:', X_val.shape, 'Test split:', X_test.shape)

Train split: (4124, 20) Val split: (1031, 20) Test split: (1289, 20)


In [72]:
%%time
import category_encoders as ce 
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline

pipeline = make_pipeline(
    ce.OneHotEncoder(use_cat_names=True),
    SimpleImputer(strategy='mean'),
    RandomForestClassifier(n_jobs=-1, random_state=42)
)

#Fit on train and score on val
pipeline.fit(X_train, y_train)

CPU times: user 43.2 s, sys: 2.57 s, total: 45.7 s
Wall time: 12.6 s


In [73]:
from sklearn.metrics import accuracy_score
y_pred = pipeline.predict(X_val)
print('Baseline Accuracy', accuracy_score(y_val, y_pred))

Baseline Accuracy 0.0533462657613967


In [74]:
X_train

,Make,Model,Year,Engine_Fuel_Type,Engine_HP,Engine_Cylinders,Transmission_Type,Driven_Wheels,Vehicle_Style,highway_MPG,city_mpg,Unknown,Hatchback,Crossover,Flex Fuel,Performance,Luxury,Hybrid,Diesel,Factory Tuner
7338,Ford,Transit Connect,2017,flex-fuel,169,4,AUTOMATIC,front wheel drive,Cargo Van,27,20,True,False,False,False,False,False,False,False,False
1540,GMC,Canyon,2016,regular unleaded,200,4,AUTOMATIC,rear wheel drive,Pickup,27,20,True,False,False,False,False,False,False,False,False
7774,Hyundai,Veloster,2016,regular unleaded,201,4,AUTOMATIC,front wheel drive,Hatchback,33,27,False,False,False,False,False,False,False,False,False
4276,Dodge,Journey,2016,regular unleaded,283,6,AUTOMATIC,all wheel drive,SUV,24,16,False,False,True,False,False,False,False,False,False
7968,Subaru,WRX,2017,premium unleaded,268,4,MANUAL,all wheel drive,Sedan,27,20,False,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7414,Chevrolet,Trax,2015,regular unleaded,138,4,AUTOMATIC,all wheel drive,SUV,31,24,False,False,True,False,False,False,False,False,False
7990,BMW,X4,2016,premium unleaded,240,4,AUTOMATIC,all wheel drive,SUV,28,20,False,False,False,False,False,False,False,False,False
2154,Mazda,CX-7,2010,premium unleaded,244,4,AUTOMATIC,all wheel drive,SUV,23,17,False,False,True,False,False,False,False,False,False
823,Nissan,Altima,2015,regular unleaded,270,6,AUTOMATIC,front wheel drive,Sedan,32,22,False,False,False,False,True,False,False,False,False


In [75]:
##Linear Model
import category_encoders as ce
import numpy as np
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Ridge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

pipeline = make_pipeline(
    ce.OneHotEncoder(use_cat_names=True), 
    SimpleImputer(),  
    StandardScaler(),
    SelectKBest(f_regression), 
    Ridge()
)

param_distributions = {
    'simpleimputer__strategy': ['mean', 'median'], 
    'selectkbest__k': range(1, len(X_train.columns)+1), 
    'ridge__alpha': [0.1, 1, 10], 
}

# If you're on Colab, decrease n_iter & cv parameters
search = RandomizedSearchCV(
    pipeline, 
    param_distributions=param_distributions, 
    n_iter=100, 
    cv=5, 
    scoring='r2', 
    verbose=10, 
    return_train_score=True, 
    n_jobs=-1
)

search.fit(X_train, y_train);

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    5.8s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    8.0s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   11.7s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   14.8s
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   20.2s
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   24.6s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:   30.9s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:   36.2s
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed:   42.8s
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:   50.1s
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed:   57.9s
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 137 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 173 tasks      | elapsed:  1

In [76]:
print('Best hyperparameters', search.best_params_)
print('Cross-validation MAE', -search.best_score_)

Best hyperparameters {'simpleimputer__strategy': 'median', 'selectkbest__k': 20, 'ridge__alpha': 0.1}
Cross-validation MAE -0.7077013121971887


In [77]:
#Cross validation
import category_encoders as ce
import numpy as np
from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

pipeline = make_pipeline(
    ce.OneHotEncoder(use_cat_names=True), 
    SimpleImputer(strategy='mean'), 
    StandardScaler(), 
    SelectKBest(f_regression, k=22), 
    Ridge(alpha=.1)
)

k = 22
scores = cross_val_score(pipeline, X_train, y_train, cv=k, 
                         scoring='r2')
print(f'MAE for {k} folds:', -scores)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: invalid value encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: invalid value encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib

MAE for 22 folds: [-0.68569567 -0.67956796 -0.71881677 -0.73026488 -0.70138524 -0.75458962
 -0.72073601 -0.68630394 -0.66709029 -0.71110262 -0.78053194 -0.71544139
 -0.72991093 -0.7479401  -0.72431816 -0.6634319  -0.7070652  -0.72302892
 -0.69485042 -0.724225   -0.71026157 -0.6997992 ]


/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: invalid value encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


In [78]:
pipeline.fit(X_train, y_train)
print('Accuracy', pipeline.score(X_val, y_val))

Accuracy 0.7112804174263667


/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: invalid value encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


In [79]:
-scores.mean()

-0.712561715323425

In [80]:
#Random Forest

from sklearn.ensemble import RandomForestRegressor

pipeline = make_pipeline(
    ce.TargetEncoder(min_samples_leaf=1, smoothing=1), 
    SimpleImputer(strategy='mean'), 
    RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=42)
)

k = 22
scores = cross_val_score(pipeline, X_train, y_train, cv=k, 
                         scoring='r2')
print(f'MAE for {k} folds:', -scores)

MAE for 22 folds: [-0.89694025 -0.87511028 -0.87459635 -0.88626603 -0.88776388 -0.87823984
 -0.8687133  -0.88564368 -0.85716359 -0.875181   -0.88330004 -0.90313844
 -0.87823906 -0.89850824 -0.91222673 -0.89022602 -0.90722744 -0.90645433
 -0.87580288 -0.86918319 -0.8954108  -0.88229969]


In [81]:
-scores.mean()

-0.885801592882464

In [82]:
from scipy.stats import randint, uniform

pipeline = make_pipeline(
    ce.TargetEncoder(), 
    SimpleImputer(), 
    RandomForestRegressor(random_state=42)
)

param_distributions = {
    'targetencoder__min_samples_leaf': randint(1, 1000), 
    
#     Remove this hyperparameter, because of an issue: 
#     https://github.com/scikit-learn-contrib/categorical-encoding/issues/184
#     'targetencoder__smoothing': uniform(1, 1000), 
    
    'simpleimputer__strategy': ['median'], 
    'randomforestregressor__n_estimators': randint(50, 500), 
    'randomforestregressor__max_depth': [5, 10, 15, 20, None], 
    'randomforestregressor__max_features': uniform(0, 1), 
}

# If you're on Colab, decrease n_iter & cv parameters
search = RandomizedSearchCV(
    pipeline, 
    param_distributions=param_distributions, 
    n_iter=10, 
    cv=16, 
    scoring='r2', 
    verbose=10, 
    return_train_score=True, 
    n_jobs=-1
)

search.fit(X_train, y_train);

Fitting 16 folds for each of 10 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    4.2s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    7.6s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    9.7s
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   15.0s
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   17.5s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:   25.7s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:   32.8s
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed:   54.7s
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 137 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed:  2.2min finished


In [83]:
print('Best hyperparameters', search.best_params_)
print('Cross-validation MAE', search.best_score_)

Best hyperparameters {'randomforestregressor__max_depth': 20, 'randomforestregressor__max_features': 0.9530951500259202, 'randomforestregressor__n_estimators': 399, 'simpleimputer__strategy': 'median', 'targetencoder__min_samples_leaf': 37}
Cross-validation MAE 0.8785348669398738


In [84]:
pipeline = make_pipeline(
    ce.TargetEncoder(), 
    SimpleImputer(), 
    RandomForestRegressor(random_state=42)
)

param_distributions = {
    #'targetencoder__min_samples_leaf': randint(1, 110), 
    'targetencoder__min_samples_leaf': 32,
#     Remove this hyperparameter, because of an issue: 
#     https://github.com/scikit-learn-contrib/categorical-encoding/issues/184
#     'targetencoder__smoothing': uniform(1, 1000), 
    
    'simpleimputer__strategy': ['median'], 
    'randomforestregressor__n_estimators': randint(1,247), 
    'randomforestregressor__max_depth': [], 
    'randomforestregressor__max_features': uniform(0.1737), 
}

# If you're on Colab, decrease n_iter & cv parameters
search = RandomizedSearchCV(
    pipeline, 
    param_distributions=param_distributions, 
    n_iter=10, 
    cv=16, 
    scoring='r2', 
    verbose=10, 
    return_train_score=True, 
    n_jobs=-1
)

#search.fit(X_train, y_train);
pipeline.fit(X_train, y_train)
print('Accuracy', pipeline.score(X_val, y_val))

Accuracy 0.8926681565763431


In [85]:
pipeline.fit(X_train, y_train)
print('Accuracy', pipeline.score(X_val, y_val))

Accuracy 0.8926681565763431
